In [2]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

# from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *

import matplotlib.pyplot as plt
%matplotlib inline

Setting environment for Local Macbook Pro


In [3]:
DETECTED_CELLS_DIR = '/home/yuncong/CSHL_cells_v2/detected_cells'

stack = 'MD589'

In [4]:
first_sec, last_sec = metadata_cache['section_limits'][stack]
s2f = metadata_cache['sections_to_filenames'][stack]

In [10]:
bar = show_progress_bar(first_sec, last_sec)

for sec in range(first_sec, last_sec+1, 10):
    
    bar.value = sec
    
    img_fn = s2f[sec]

    if img_fn in ['Nonexisting', 'Rescan', 'Placeholder']:
        continue

    input_img_fp = DETECTED_CELLS_DIR + '/%(stack)s/%(img_fn)s/%(img_fn)s_image_inverted.jpg' % dict(stack=stack, img_fn=img_fn)
    
    if not os.path.exists(input_img_fp):
        # download from Gordon
        execute_command('scp -r yuncong@gcn:/home/yuncong/csd395/CSHL_cells_v2/detected_cells/%(stack)s/%(img_fn)s/ %(DETECTED_CELLS_DIR)s/%(stack)s/' % \
                       dict(stack=stack, img_fn=img_fn, DETECTED_CELLS_DIR=DETECTED_CELLS_DIR))
        
    
    img = imread(input_img_fp)
    img_h, img_w = img.shape[:2]

    input_fps = [os.path.splitext(input_img_fp)[0] + '_%02d.tif' % tile_idx for tile_idx in range(12)]
    
    #############
    # Farsight
    #############

#     output_fps = [os.path.splitext(input_fp)[0] + '_labeled_farsight.tif' for input_fp in input_fps]

#     FARSIGHT_EXEC = '/home/yuncong/Farsight-0.4.4-Linux/bin/segment_nuclei'

#     # # Use without parameters

#     # for input_fp, output_fp in zip(input_fps, output_fps):
#     #     execute_command(FARSIGHT_EXEC + ' %(input_fp)s %(output_fp)s' % \
#     #                    dict(input_fp=input_fp,
#     #                         output_fp=output_fp))

#     # Use with parameters

#     FARSIGHT_PARAMS_FP = '/home/yuncong/CSHL_cells_v2/farsight_parameters.txt'

#     with open(FARSIGHT_PARAMS_FP, 'w') as f:
#         f.write('! Segmentation parameters File\n! All parameters are case sensitive\n')
#         f.write('min_object_size 100\n')

#     for input_fp, output_fp in zip(input_fps, output_fps):

#         execute_command(FARSIGHT_EXEC + ' %(input_fp)s %(output_fp)s %(params)s' % \
#                    dict(input_fp=input_fp,
#                         output_fp=output_fp,
#                         params=FARSIGHT_PARAMS_FP))

#     for input_fp in input_fps:
#         prefix = os.path.splitext(input_fp)[0]
#         labelmap = imread(prefix + '_labeled_farsight.tif')
#         bp.pack_ndarray_file(labelmap, prefix + '_labelmap_farsight.bp')
#         execute_command('rm ' + prefix + '_seg_final.dat')
#         execute_command('rm ' + prefix + '_seedPoints.txt')
#         execute_command('rm ' + prefix + '_labeled_farsight.tif')

    ###############
    # CellProfiler
    ###############

    CELLPROFILER_EXEC = 'cellprofiler' # /usr/local/bin/cellprofiler
    CELLPROFILER_PIPELINE_FP = '/home/yuncong/CSHL_cells_v2/SegmentCells.cppipe'

    with open('/tmp/cellprofiler_filelist.txt', 'w') as f:
        for input_fp in input_fps:
            f.write(input_fp + '\n')

    t = time.time()

    execute_command(CELLPROFILER_EXEC + ' -c --file-list=/tmp/cellprofiler_filelist.txt -p %(pipeline_fp)s' % dict(pipeline_fp=CELLPROFILER_PIPELINE_FP))

    sys.stderr.write('Cell profiler: %.2f seconds.\n' % (time.time()-t))

    from scipy.io import loadmat

    for input_fp in input_fps:
        prefix = os.path.splitext(input_fp)[0]
        labelmap = loadmat(prefix + '_labeled.mat')['Image']
        bp.pack_ndarray_file(labelmap, prefix + '_labelmap_cellprofiler.bp')
        execute_command('rm ' + prefix + '_labeled.mat')

    ##########
    # Mosaic
    ##########

    # tile_h, tile_w = (5000, 5000)

    origins = []
    for iy, y0 in enumerate(np.arange(0, img_h, 5000)):
        for ix, x0 in enumerate(np.arange(0, img_w, 5000)):
            origins.append((x0, y0))

    alg = 'cellprofiler'
    # alg = 'farsight'

    big_labelmap = np.zeros((img_h, img_w), dtype=np.int64)

    n = 0

    for i, input_fp in enumerate(input_fps):
        prefix = os.path.splitext(input_fp)[0]
        labelmap = bp.unpack_ndarray_file(prefix + '_labelmap_%(alg)s.bp' % dict(alg=alg)).astype(np.int64)

        x0, y0 = origins[i]
        big_labelmap[y0:y0+5000, x0:x0+5000][labelmap != 0] = labelmap[labelmap != 0] + n
        n += labelmap.max()

    bp.pack_ndarray_file(big_labelmap, os.path.splitext(input_img_fp)[0] + '_labelmap_%(alg)s.bp' % dict(alg=alg))

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


cellprofiler -c --file-list=/tmp/cellprofiler_filelist.txt -p /home/yuncong/CSHL_cells_v2/SegmentCells.cppipe


Child returned 0
Cell profiler: 307.44 seconds.
Child returned 0
Child returned 0
Child returned 0
Child returned 0


rm /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N16-2015.07.30-17.03.43_MD589_3_0048/MD589-N16-2015.07.30-17.03.43_MD589_3_0048_image_inverted_00_labeled.mat
rm /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N16-2015.07.30-17.03.43_MD589_3_0048/MD589-N16-2015.07.30-17.03.43_MD589_3_0048_image_inverted_01_labeled.mat
rm /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N16-2015.07.30-17.03.43_MD589_3_0048/MD589-N16-2015.07.30-17.03.43_MD589_3_0048_image_inverted_02_labeled.mat
rm /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N16-2015.07.30-17.03.43_MD589_3_0048/MD589-N16-2015.07.30-17.03.43_MD589_3_0048_image_inverted_03_labeled.mat
rm /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N16-2015.07.30-17.03.43_MD589_3_0048/MD589-N16-2015.07.30-17.03.43_MD589_3_0048_image_inverted_04_labeled.mat


Child returned 0
Child returned 0
Child returned 0
Child returned 0
Child returned 0
Child returned 0
Child returned 0
Child returned 0


...

Child returned 0
Cell profiler: 253.64 seconds.
Child returned 0
Child returned 0
Child returned 0
Child returned 0


rm /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N68-2015.07.30-20.43.04_MD589_2_0203/MD589-N68-2015.07.30-20.43.04_MD589_2_0203_image_inverted_00_labeled.mat
rm /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N68-2015.07.30-20.43.04_MD589_2_0203/MD589-N68-2015.07.30-20.43.04_MD589_2_0203_image_inverted_01_labeled.mat
rm /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N68-2015.07.30-20.43.04_MD589_2_0203/MD589-N68-2015.07.30-20.43.04_MD589_2_0203_image_inverted_02_labeled.mat
rm /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N68-2015.07.30-20.43.04_MD589_2_0203/MD589-N68-2015.07.30-20.43.04_MD589_2_0203_image_inverted_03_labeled.mat
rm /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N68-2015.07.30-20.43.04_MD589_2_0203/MD589-N68-2015.07.30-20.43.04_MD589_2_0203_image_inverted_04_labeled.mat


Child returned 0
Child returned 0
Child returned 0
Child returned 0
Child returned 0
Child returned 0


rm /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N68-2015.07.30-20.43.04_MD589_2_0203/MD589-N68-2015.07.30-20.43.04_MD589_2_0203_image_inverted_05_labeled.mat
rm /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N68-2015.07.30-20.43.04_MD589_2_0203/MD589-N68-2015.07.30-20.43.04_MD589_2_0203_image_inverted_06_labeled.mat
rm /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N68-2015.07.30-20.43.04_MD589_2_0203/MD589-N68-2015.07.30-20.43.04_MD589_2_0203_image_inverted_07_labeled.mat
rm /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N68-2015.07.30-20.43.04_MD589_2_0203/MD589-N68-2015.07.30-20.43.04_MD589_2_0203_image_inverted_08_labeled.mat
rm /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N68-2015.07.30-20.43.04_MD589_2_0203/MD589-N68-2015.07.30-20.43.04_MD589_2_0203_image_inverted_09_labeled.mat
rm /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N68-2015.07.30-20.43.04_MD589_2_0203/MD589-N68-2015.07.30-20.43.04_MD589_2_0203_image_inverted_10_labe

Child returned 0
Child returned 0


In [11]:
# Upload generated labelmap bp files

for sec in range(first_sec, last_sec+1, 10):
        
    img_fn = s2f[sec]

    if img_fn in ['Nonexisting', 'Rescan', 'Placeholder']:
        continue

    input_img_fp = DETECTED_CELLS_DIR + '/%(stack)s/%(img_fn)s/%(img_fn)s_image_inverted.jpg' % dict(stack=stack, img_fn=img_fn)
    
    labelmap_fp = os.path.splitext(input_img_fp)[0] + '_labelmap_%(alg)s.bp' % dict(alg=alg)
    
    # upload to Gordon
    execute_command('scp %(labelmap_fp)s yuncong@gcn:/home/yuncong/csd395/CSHL_cells_v2/detected_cells/%(stack)s/%(img_fn)s/ ' % \
                   dict(labelmap_fp=labelmap_fp, stack=stack, img_fn=img_fn))

Setting environment for Local Macbook Pro
scp /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N16-2015.07.30-17.03.43_MD589_3_0048/MD589-N16-2015.07.30-17.03.43_MD589_3_0048_image_inverted_labelmap_cellprofiler.bp yuncong@gcn:/home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-N16-2015.07.30-17.03.43_MD589_3_0048/ 


Child returned 0


scp /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N18-2015.07.30-17.11.24_MD589_2_0053/MD589-N18-2015.07.30-17.11.24_MD589_2_0053_image_inverted_labelmap_cellprofiler.bp yuncong@gcn:/home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-N18-2015.07.30-17.11.24_MD589_2_0053/ 


Child returned 0


scp /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N20-2015.07.30-17.19.14_MD589_1_0058/MD589-N20-2015.07.30-17.19.14_MD589_1_0058_image_inverted_labelmap_cellprofiler.bp yuncong@gcn:/home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-N20-2015.07.30-17.19.14_MD589_1_0058/ 


...

Child returned 0


scp /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N66-2015.07.30-20.35.16_MD589_3_0198/MD589-N66-2015.07.30-20.35.16_MD589_3_0198_image_inverted_labelmap_cellprofiler.bp yuncong@gcn:/home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-N66-2015.07.30-20.35.16_MD589_3_0198/ 


Child returned 0


scp /home/yuncong/CSHL_cells_v2/detected_cells/MD589/MD589-N68-2015.07.30-20.43.04_MD589_2_0203/MD589-N68-2015.07.30-20.43.04_MD589_2_0203_image_inverted_labelmap_cellprofiler.bp yuncong@gcn:/home/yuncong/csd395/CSHL_cells_v2/detected_cells/MD589/MD589-N68-2015.07.30-20.43.04_MD589_2_0203/ 


Child returned 0


In [14]:
viz = img_as_ubyte(label2rgb(big_labelmap, bg_label=0, bg_color=(0, 0, 0)))

/usr/local/lib/python2.7/dist-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [15]:
cv2.imwrite(os.path.splitext(input_img_fp)[0] + '_labelmap_%(alg)s.png' % dict(alg=alg), viz);